# Project 2019 - Programming for Data Analysis

# Simulation Dataset - Installed Base Dataset

## Introduction
The following assignment concerns the numpy.random package in Python 3. I have created a Jupyter notebook explaining the use of the package, including detailed explanations of six of the distributions provided for in the package.

## Problem statement

The objective of this project is to create a data set by simulating a real-world phenomenon, of my choosing.

* Instead of collecting data, I model and synthesise the data using Python packages, such as numpy.random.

Specifically, in this project you should:

* I simulate two hundred data points across four different variables.
* I investigate the types of variables involved, their likely distributions, and their relationships with each other.
* I simulate a data set as closely matching their properties as possible.
* I detail my research and implement the simulation in a Jupyter notebook 
* The final dataset itself is displayed in an output cell within the notebook.



## About My Dataset
The objective of my dataset is to simulate typical data about a companys installed base. According to Kurvinen (2017), the typical data on install base includes a listing of products which are installed at a given customer site. Futhermore, it can include additional variables such as serial numbers, hardware and software revisions, warranty and service contracts. 

Installed base data can be used by many departments in an organisation, from field service engineers, sales, spare parts planners, quality etc . The dataset helps to answer questions such as:

* What is the current configuration of the product to be serviced?
* Where is the faulty product physically located and where is the part to be replaced located?
* Is the unit covered under warranty or service contract?
* When was the unit installed, upgraded and/or last serviced?

![Install Base Image](https://cdn.myonlinestore.eu/945f2dab-6be1-11e9-a722-44a8421b9960/images/World%20map%20installed%20base.png)

## Basic Description of the Dataset

The dataset contains the following data

1. Part Number: 5 Digit identifier for each product installed. 
2. Serial Numbers: Unique ID's for each unit sold. There are 200 serial numbers.
3. Installation Date: Assume all units sold require a start-up / installation by a field service engineer, who records this date on the company CRM system (Saleforce.com). The dataset was created in January 2017 and spans 36 months.
4. Factory Warranty Expiration Date: Factory Warranty usually expires 1 year after installation.
5. Extended Warranty: Number of years of extended warranty cover purchased.
6. 

## Variables Types
 
 1. Python Objects  
 Serial numbers are usually unique alphanumeric strings, usually of a fixed length, so I decided to use UUID function in python to generate random unique ID's which have similar properties to serial numbers. The UUID module provides "immutable UUID objects (the UUID class) and the functions uuid1(), uuid3(), uuid4(), uuid5() for generating version 1, 3, 4, and 5 UUIDs as specified in RFC 4122" (The Python Foundation, 2019). The dtype used to store these objects, is Python Object.   
 
 2. datetime64
 Datetime64 is a NumPy data types which support datetime functionality (The Scipy Community, 2017). This datatype will be used to store datetime variables 'install date' & 'factory warranty'.
 
 3. Int32
 This datatype is a 32bit integer, which will be used for variable 'extended warranty'.
 
 

## 1. Defining the main parameters

I begin by importing all the necessary dependencies and defining the main parameters mentioned above (200 serial number, 36 months, starting month in January 2017).

In [14]:
# importing all the libraries
import pandas as pd
import numpy as np
import uuid
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [15]:
# Define the main parameters dataset. 

# number of serial numbers which are contained in the install base dataset
num_serial_num = 200

# number of months since first unit was installed
num_months = 36

# starting month when units first installed
start_month = '2017-01-01'

## 2. Simulate List of Part Numbers
Part numbers can be classified as categorical data. 

Typically, any data attribute which is categorical in nature represents discrete values which belong to a specific finite set of categories or classes. These are also often known as classes or labels in the context of attributes or variables which are to be predicted by a model (popularly known as response variables). These discrete values can be text or numeric in nature (or even unstructured data like images!). There are two major classes of categorical data, nominal and ordinal.
In any nominal categorical data attribute, there is no concept of ordering amongst the values of that attribute. Consider a simple example of weather categories, as depicted in the following figure. We can see that we have six major classes or categories in this particular scenario without any concept or notion of order (windy doesn’t always occur before sunny nor is it smaller or bigger than sunny). Sarkar, 2018

In [30]:
# Use NumPy Random Randint Function to generate 200 random part numbers

# 5 Digit Part Numbers with ID # in range of 10000 to 99999
low, high, size = (10000 , 99999 , 200) # Define parameters for numpy.random.randint() function

np.random.seed(2) # use random seed generator

# Create a Pandas Dataframe - # solution adapted from https://stackoverflow.com/a/23671779
# Create 2D array of 200 sets of 5 numbers
parts = pd.DataFrame(np.random.randint(low, high, size), columns=['parts'])
# Display first 5 rows
parts.head()

,parts
0,99256
1,82173
2,54566
3,41019
4,94434


## 3. Create Serial Numbers

In [32]:
# generating 200 serial numbers using UUID function
# Adapted from https://towardsdatascience.com/generating-product-usage-data-from-scratch-with-pandas-319487590c6d

# create an 1D array called serial, using uuid4 function to generate 200 random UUID. 
serial = pd.Series([str(uuid.uuid4()) for i in range(0,num_serial_num)])
# create 2D dataframe called installbase and insert array serial
installbase = pd.DataFrame()
# insert array serial into installbase dataframe
installbase['serial number'] = pd.Series(serial)
# display dataframe (scrolling)
installbase


,serial number
0,ebb7e6a5-b192-47fc-9db8-d90e0cb7a1f9
1,62d5dd29-cdfa-422f-b29c-96744b33f71d
2,ce555016-2871-431b-8b2f-e29babce71e2
3,0bb55b50-4b94-48f5-929a-ce825030d946
4,f77c0a30-22cf-405f-a8af-ebf74a84759b
5,24bb7dbd-36d1-49c4-b48b-84645c4c423e
6,1cbe7625-220d-496d-adf5-a41248fcfaed
7,a20c6c5a-3bdb-4b23-b79b-19574c6e9581
8,726d6016-8382-403a-8360-b52db94cab6d
9,75f1957b-d5cd-477d-9533-97212bc50e41


In [33]:
# Adding part number column to the dataframe using pandas concat function
# Adapted from https://towardsdatascience.com/generating-product-usage-data-from-scratch-with-pandas-319487590c6d

# Combine 2 pandas series in the dataframe, resetting the index without inserting it as a column in the new DataFrame.
installbase['part number'] = pd.concat([parts], axis=0).reset_index().drop('index', 1)
installbase

,serial number,part number
0,ebb7e6a5-b192-47fc-9db8-d90e0cb7a1f9,99256
1,62d5dd29-cdfa-422f-b29c-96744b33f71d,82173
2,ce555016-2871-431b-8b2f-e29babce71e2,54566
3,0bb55b50-4b94-48f5-929a-ce825030d946,41019
4,f77c0a30-22cf-405f-a8af-ebf74a84759b,94434
5,24bb7dbd-36d1-49c4-b48b-84645c4c423e,43867
6,1cbe7625-220d-496d-adf5-a41248fcfaed,90232
7,a20c6c5a-3bdb-4b23-b79b-19574c6e9581,54295
8,726d6016-8382-403a-8360-b52db94cab6d,63922
9,75f1957b-d5cd-477d-9533-97212bc50e41,43201


The output is a dataframe 200 rows and two columns, serial number and part number. 

## 4. Generating Installation Date

I use the pandas.to_datetime function to output a range of datetime values which simulate a range of installation dates. Then I select a random sample of these dates to populate the dataset using np.random.choice function.

In [19]:
# Create Date Range using the specified parameters (starting month, number of months).
# Adapted from https://towardsdatascience.com/generating-product-usage-data-from-scratch-with-pandas-319487590c6d

# reseting the index without inserting it as a column in the new DataFrame - https://www.geeksforgeeks.org/python-pandas-series-reset_index/
installbase = installbase.reset_index().drop('index', 1)

# defining range of installation dates: starting month 
start_month_ts = pd.to_datetime(start_month)
# define end month as start month plus 36 months, using relativedata utility https://dateutil.readthedocs.io/en/stable/relativedelta.html
end_month_ts = start_month_ts + relativedelta(months=+num_months - 1)

# making a Series out of the starting and ending month
months = pd.Series(pd.date_range(start_month_ts, end_month_ts, freq='MS'))
# Display start and end month
str(start_month_ts), str(end_month_ts)

('2017-01-01 00:00:00', '2019-12-01 00:00:00')

This argument uses the specified parameters (starting month, number of months) to generate a range of dates from January 2017 to December 2019.

In [20]:
# Select random installation dates from range of dates created above

# use random seed generator
np.random.seed(2) # use random seed generator
# create a series called installdate whoch contain 200 dates chosen at random from  the daterange "months" using random.choice() function 
installdate = pd.Series(np.random.choice(months, size=num_serial_num))
# display first 5 rows
installdate.head()

0   2018-04-01
1   2017-09-01
2   2018-11-01
3   2018-07-01
4   2017-12-01
dtype: datetime64[ns]

In [21]:
#  Adding installdate column to the dataframe

# Combine installdate series into the installbase dataframe, resetting the index without inserting it as a column in the new DataFrame.
installbase['install date'] = pd.concat([installdate] * num_serial_num, axis=0).reset_index().drop('index', 1)
#diplay the dataframe
installbase

,serial,part number,install date
0,69ec6fd3-dcca-47c4-ae53-0cdcdcd802bd,22849,2018-04-01
1,bca3d1d1-ade3-4b98-afb1-34976a237cde,93937,2017-09-01
2,3c2f5a8f-dff9-452f-b6c6-66eab5560ca7,89295,2018-11-01
3,cde12a12-3117-4aca-acb3-5c66736a9492,69404,2018-07-01
4,aadaa0c1-0b8b-4170-8133-c6df4f3fcf8c,85306,2017-12-01
5,e7bcce6a-4e04-4e6b-8981-1ff716857d57,65810,2017-08-01
6,17d54f1d-b83f-4ec8-9374-7053bc8849f4,98136,2019-11-01
7,978e7df2-76c9-4ff0-b799-da88f9528e66,91234,2019-08-01
8,4aaea158-8f41-4931-8555-41b7470f6c79,13111,2017-12-01
9,743e54c1-28a3-4d25-969e-b4dbca052945,85906,2018-10-01


In [22]:
factorywarrantyexpiry = (installdate + np.timedelta64(1, 'Y'))
factorywarrantyexpiry

0     2019-04-01 05:49:12
1     2018-09-01 05:49:12
2     2019-11-01 05:49:12
3     2019-07-01 05:49:12
4     2018-12-01 05:49:12
5     2018-08-01 05:49:12
6     2020-10-31 05:49:12
7     2020-07-31 05:49:12
8     2018-12-01 05:49:12
9     2019-10-01 05:49:12
10    2020-07-31 05:49:12
11    2020-02-29 05:49:12
12    2019-09-01 05:49:12
13    2018-04-01 05:49:12
14    2018-05-01 05:49:12
15    2020-09-30 05:49:12
16    2018-04-01 05:49:12
17    2018-06-01 05:49:12
18    2020-01-01 05:49:12
19    2018-05-01 05:49:12
20    2018-07-01 05:49:12
21    2020-07-31 05:49:12
22    2019-08-01 05:49:12
23    2020-07-31 05:49:12
24    2018-03-01 05:49:12
25    2019-05-01 05:49:12
26    2019-01-01 05:49:12
27    2018-05-01 05:49:12
28    2020-02-29 05:49:12
29    2019-04-01 05:49:12
              ...        
170   2019-10-01 05:49:12
171   2019-08-01 05:49:12
172   2018-05-01 05:49:12
173   2020-10-31 05:49:12
174   2019-01-01 05:49:12
175   2019-04-01 05:49:12
176   2019-10-01 05:49:12
177   2018-0

In [23]:
installbase['factory warranty'] = pd.concat([factorywarrantyexpiry], axis=0).reset_index().drop('index', 1)
installbase.head()

,serial,part number,install date,factory warranty
0,69ec6fd3-dcca-47c4-ae53-0cdcdcd802bd,22849,2018-04-01,2019-04-01 05:49:12
1,bca3d1d1-ade3-4b98-afb1-34976a237cde,93937,2017-09-01,2018-09-01 05:49:12
2,3c2f5a8f-dff9-452f-b6c6-66eab5560ca7,89295,2018-11-01,2019-11-01 05:49:12
3,cde12a12-3117-4aca-acb3-5c66736a9492,69404,2018-07-01,2019-07-01 05:49:12
4,aadaa0c1-0b8b-4170-8133-c6df4f3fcf8c,85306,2017-12-01,2018-12-01 05:49:12


In [24]:
# Customer have a choice of not taking out extended warranty or purchasing an additional 1, 2 or 3 year extended warranty
num_warranty = 3  

# assign extended warranty to users randomly (when did the user first use the product?)
warranty = pd.DataFrame()
warranty['warranty extension'] = np.random.randint(low=0, high=num_warranty, size=num_serial_num)
warranty.sample(5)



,warranty extension
30,1
54,2
71,1
162,2
119,0


In [25]:
installbase['extended warranty'] = pd.concat([warranty], axis=0).reset_index().drop('index', 1)
installbase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
serial               200 non-null object
part number          200 non-null int32
install date         200 non-null datetime64[ns]
factory warranty     200 non-null datetime64[ns]
extended warranty    200 non-null int32
dtypes: datetime64[ns](2), int32(2), object(1)
memory usage: 6.3+ KB


## 4. Generating categorical features

In [26]:
# Defining the variables
platforms = ['iOS', 'Android']
countries = ['IE', 'GB', 'NL', 'FR', 'DE', 'BE', 'DK']
service_contract = [False, True]

4.1. Generating categorical feature weights
Defining weights for the likelihood of a categorical feature associated with an individual unit.

## References
1. Kurvinen, M (2017) *INSTALLED BASE AND TRACEABILITY* [Online] Available at: http://sd-ize.com/installed-base.html[Accessed 1 December 2019].
2. Python Software Foundation (2019) *UUID objects according to RFC 4122* [Online] Available at https://docs.python.org/2/library/uuid.html [Accessed 1 December 2019]
3. Osolnik, J (2017) *Simulating product usage data with Pandas* [Online] Available at https://towardsdatascience.com/generating-product-usage-data-from-scratch-with-pandas-319487590c6d [Accessed 1 December 2019]
4. The Scipy Community (2017) *Datetimes and Timedeltas* [Online] Available at https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.datetime.html [Accessed 3 December 2019]
5. Sarkar, D (2018) Categorical Data [Online] Available at: https://towardsdatascience.com/understanding-feature-engineering-part-2-categorical-data-f54324193e63 [Accessed 21 November 2019].

